In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from g2p_en import G2p
import re

from basicOperations.manifoldOperations import matrixDistance, frechetMean
import torch.nn.utils as utils

from rnn import euclideanRnn
import math

import pickle
import Levenshtein
import os

import matplotlib.pyplot as plt

In [ ]:
"""
Proof for table 1, figure 2, and figure 3. 
"""

In [ ]:
"""
Train LARGE-VOCAB EMG-to-phoneme conversion.

For description of the data, please see largeVocabDataVisualization.ipynb

Unlike data SMALL-VOCAB, there are no timestamps between words within a sentence. 

Given a sentence, you decode it fully using CTC loss. The pipeline resembles standard speech-to-text (ASR) techniques.
"""

"""https://pypi.org/project/Levenshtein/ - install this Lev distance."""

In [2]:
"""
Open Data.
"""

with open("DATA/dataLargeVocab.pkl", "rb") as file:
    DATA = pickle.load(file)

with open("DATA/labelsLargeVocab.pkl", "rb") as file:
    LABELS = pickle.load(file)

In [3]:
for i in range(len(LABELS)):
    print(LABELS[i])

nuclear rockets can destroy airfields with ease
the best way to learn is to solve extra problems
the spray will be used in first division matches next season
our experiments positive outcome was unexpected
alimony harms a divorced mans wealth
she uses both names interchangeably
the misquote was retracted with an apology
critics fear the erosion of consumer protections and environmental standards
her lips moist and parted spoke his name
how do they turn out later
the eastern coast is a place for pure pleasure and excitement
please make your decision wisely to visit the beach
she encouraged her children to make their own halloween costumes
cleaned cloth must be protected against the redeposition of dispersed soil
both figures would go higher in later years
she always jokes about too much garlic in his food
if people were more generous there would be no need for welfare
the dimensions of these waves dwarf all our usual standards of measurement
the knifelike pain in his groin nearly brough

In [4]:
"""
English phoneme definitions.
"""
tok2id = {}
with open("DATA/ckptsLargeVocab/lang_phone/tokens.txt") as f:
    for line in f:
        s, i = line.strip().split()
        i = int(i)
        if s == "<eps>" or s.startswith("#"):
            continue
        tok2id[s] = i
PHONE_DEF = tok2id


def phoneToId(p):
    return PHONE_DEF[p]

g2p = G2p()

In [5]:
"""
Phonemize the sentences.
"""

phonemizedSentences = []

for i in range(len(LABELS)):
    phones = []
    for p in g2p(LABELS[i]): 
        p = re.sub(r'[0-9]', '', p)   
        if re.match(r'[A-Z]+', p): 
            phones.append(p)
    phonemizedSentences.append(phones)

In [6]:
"""
Convert phone-to-indices using look-up dictionary PHONE_DEF.
"""

phoneIndexedSentences = []
for i in range(len(phonemizedSentences)):
    current = phonemizedSentences[i]
    phoneID = []
    for j in range(len(current)):
        phoneID.append(phoneToId(current[j]))
    phoneIndexedSentences.append(phoneID)

In [7]:
def tokenIdToClassIdx(tokenId: int) -> int:
    return tokenId - 1   

def phoneSeqToClassIdxSeq(phoneSeq):
    return [tokenIdToClassIdx(PHONE_DEF[p]) for p in phoneSeq]

classIndexedSentences = [phoneSeqToClassIdxSeq(seq) for seq in phonemizedSentences]

In [8]:
"""
Pad the phone transcribed sentences to a common length (to be used with CTC loss).
"""

phonemizedLabels = np.zeros((len(classIndexedSentences), 76)) - 1
for i in range(len(classIndexedSentences)):
    phonemizedLabels[i, 0:len(classIndexedSentences[i])] = classIndexedSentences[i]

labelLengths = np.zeros((len(classIndexedSentences)))
for i in range(len(classIndexedSentences)):
    labelLengths[i] = len(classIndexedSentences[i])

In [9]:
"""
z-normalize the data along the time dimension.
"""

normDATA = []
for i in range(len(DATA)):
    Mean = np.mean(DATA[i], axis = -1)
    Std = np.std(DATA[i], axis = -1)
    normDATA.append((DATA[i] - Mean[..., np.newaxis])/Std[..., np.newaxis])

In [10]:
"""
Slice the matrices into 50ms segments with a step size of 20ms.
"""

slicedMatrices = []
for j in range(len(normDATA)):
    collect = []
    stepSize = 100 
    windowSize = 125
    dataLength = normDATA[j].shape[1]
    numIters = (dataLength - windowSize) // stepSize + 1
       
    for i in range(numIters):
        where = i * stepSize + windowSize
        start = where - windowSize
        End = where + windowSize
        temp = 1/(2 * windowSize) * (normDATA[j][:, start:End] @ normDATA[j][:, start:End].T)
        collect.append(0.9 * temp + 0.1 * np.trace(temp) * np.eye(31))
    slicedMatrices.append(collect)

In [11]:
""" Diag = TRUE or FALSE. Raw SPD matrices or approximately diagonalized?"""
DIAG = True

In [12]:
matricesForMean = []
for i in range(9000):
    for j in range(len(slicedMatrices[i])):
        matricesForMean.append(slicedMatrices[i][j])

matricesForMean = np.array(matricesForMean)
manifoldMean = frechetMean()

MEAN = manifoldMean.mean(matricesForMean.reshape(-1, 31, 31))
eigenvalues, eigenvectors = np.linalg.eig(MEAN)

identityMatrix = np.eye(31)
afterMatrices = np.tile(identityMatrix, (len(slicedMatrices), 409, 1, 1)) 
inputLengths = np.zeros((len(slicedMatrices)))
for i in range(len(slicedMatrices)):
    for j in range(len(slicedMatrices[i])):
        if DIAG:
            temp = eigenvectors.T @ slicedMatrices[i][j] @ eigenvectors
        else:
            temp = slicedMatrices[i][j]
        afterMatrices[i, j] = temp
    inputLengths[i] = len(slicedMatrices[i])

In [13]:
class BaseDataset(Dataset):
    def __init__(self, data, labels, inputLength, targetLength):
        self.data = data 
        self.labels = labels
        self.targetLength = targetLength
        self.inputLength = inputLength

    def __getitem__(self, index):
        inputSeq = self.data[index].astype('float32')  
        targetSeq = self.labels[index]
        inputLength = int(self.inputLength[index])
        targetLength = int(self.targetLength[index])
        return inputSeq, targetSeq, inputLength, targetLength

    def __len__(self):
        return len(self.data)

In [14]:
"""
Train-validation-test split.
"""

trainFeatures = afterMatrices[:8000]
trainLabels = phonemizedLabels[:8000]
trainLabelLengths = labelLengths[:8000]
trainInputLengths = inputLengths[:8000]

valFeatures = afterMatrices[8000:9000]
valLabels = phonemizedLabels[8000:9000]
valLabelLengths = labelLengths[8000:9000]
valInputLengths = inputLengths[8000:9000]

testFeatures = afterMatrices[9000:]
testLabels = phonemizedLabels[9000:]
testLabelLengths = labelLengths[9000:]
testInputLengths = inputLengths[9000:]

In [15]:
trainDataset = BaseDataset(trainFeatures, trainLabels, trainInputLengths, trainLabelLengths)
valDataset = BaseDataset(valFeatures, valLabels, valInputLengths, valLabelLengths)
testDataset = BaseDataset(testFeatures, testLabels, testInputLengths, testLabelLengths)

trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
valDataloader = DataLoader(valDataset, batch_size = 32, shuffle = False)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [16]:
"""
To replicate the PER (phoneme error rate) for various model sizes and layers, change the variable here:
euclideanRnn.RnnNet(40, modelHiddenDimension = 25, device, numLayers = 3).to(device)
"""

dev = "cuda:0"
device = torch.device(dev)

numberEpochs = 100

model = euclideanRnn.RnnNet(41, 25, device, numLayers = 3).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)
lossFunction = nn.CTCLoss(blank = 40, zero_infinity = True)
rnnOptimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-3)

6348591


In [19]:
def testOperation(model, device, testLoader, Loss):
    model.eval()
    totalLoss = 0
    Outputs = []
    with torch.no_grad():
        for inputs, targets, inputLengths, targetLengths in testLoader:
            inputs, targets = inputs.to(device), targets.to(device)
            inputLengths, targetLengths = inputLengths.to(device), targetLengths.to(device)
            
            outputs = model(inputs, inputLengths.cpu()) 

            loss = Loss(outputs, targets, inputLengths, targetLengths)
            totalLoss += loss.item()
            Outputs.append(outputs.transpose(0, 1))

    return Outputs, totalLoss / len(testLoader)

In [20]:
"""
Simple beam-search algorithm.
"""

def ctcPrefixBeamSearch(
    logProbs,
    testLen = None,
    beamSize = 5,
    blank = 40,
    topk = None,
    allowDoubles = True,
):
    
    lp = np.asarray(logProbs)
    Ttotal, V = lp.shape
    T = Ttotal if testLen is None else int(min(testLen, Ttotal))

    beams = {(): (0.0, -np.inf)}

    def add(store, seq, addPb, addPnb):
        if seq in store:
            pb, pnb = store[seq]
            if addPb  != -np.inf: pb  = np.logaddexp(pb,  addPb)
            if addPnb != -np.inf: pnb = np.logaddexp(pnb, addPnb)
            store[seq] = (pb, pnb)
        else:
            store[seq] = (addPb, addPnb)

    for t in range(T):
        row = lp[t] 
        new = {}

        if topk is not None and topk < V:
            cand = np.argpartition(row, -topk)[-topk:]
            if blank not in cand:
                worstIdx = cand[np.argmin(row[cand])]
                cand[cand == worstIdx] = blank
        else:
            cand = range(V)

        for seq, (pb, pnb) in beams.items():
            add(new, seq, np.logaddexp(pb, pnb) + row[blank], -np.inf)

            last = seq[-1] if seq else None

            for c in cand:
                if c == blank:
                    continue
                pC = row[c]

                if c == last:
            
                    add(new, seq, -np.inf, pnb + pC)

                    if allowDoubles:
                        add(new, seq + (c,), -np.inf, pb + pC)
                else:
                    add(new, seq + (c,), -np.inf, np.logaddexp(pb, pnb) + pC)

        if len(new) > beamSize:
            items = sorted(new.items(),
                           key = lambda kv: np.logaddexp(*kv[1]),
                           reverse = True)[:beamSize]
            beams = dict(items)
        else:
            beams = new

    bestSeq = max(beams.items(), key = lambda kv: np.logaddexp(*kv[1]))[0]
    return bestSeq

def findClosestTranscription(decodedTranscript, phoneticTranscription):
    
    dist = Levenshtein.distance(decodedTranscript, phoneticTranscription)

    return dist

In [21]:
modelWeight = torch.load("ckpts/largeVocab/" + str(35)  + '.pt', weights_only = True)
#modelWeight = torch.load("DATA/ckptsLargeVocab/ckptWithoutSpaces.pt", weights_only = True)
model.load_state_dict(modelWeight)
output, testLoss = testOperation(model, device, testDataloader, lossFunction)

print("TEST LOSS: ", testLoss)

TEST LOSS:  1.9452395823694044


In [22]:
outs = []
for o in output:
    for oo in o:
        outs.append(oo)

In [23]:
PHONE_DEF1 = {}
for k, v in PHONE_DEF.items():
    PHONE_DEF1[v - 1] = k

In [24]:
LEVS = []
decodedOut = []
for i in range(1970):
    decodedSymbols = ctcPrefixBeamSearch(outs[i].cpu().numpy(), testInputLengths[i]) 
    phoneOut = []
    for j in range(len(decodedSymbols)):
        phoneOut.append(PHONE_DEF1[decodedSymbols[j]])
    decodedOut.append(phoneOut)

In [25]:
levs = []
phoneLENGTHS = []
for i in range(len(decodedOut)):
    phoneLENGTHS.append(len(phonemizedSentences[9000 + i]))
    levs.append(findClosestTranscription(decodedOut[i], phonemizedSentences[9000 + i]))
LEVS.append(np.mean(levs))

In [26]:
print("Mean length of sentences: ", np.mean(phoneLENGTHS))
print("Mean phoneme error rate (insertion errors + deletion errors + substitution errors): ", np.mean(levs))
print("Percent phoneme error: ", np.mean(levs)/np.mean(phoneLENGTHS))

Mean length of sentences:  19.590862944162435
Mean phoneme error rate (insertion errors + deletion errors + substitution errors):  10.213197969543147
Percent phoneme error:  0.5213245582214853


In [27]:
"""
Sort the decoded sentences from best-to-worst. Display 100 best decoded sentences.
"""

indices = np.argsort(np.array(levs)/np.array(phoneLENGTHS))
print(indices[:100])

[ 374  452 1619   59  991  669 1214  536 1491  706 1222  202 1751 1346
 1767  170  564  693 1381 1179   87  288 1119  841 1135  778  173  852
 1620 1555 1898  762 1341  602 1807 1633  620  810  813 1056  973  797
 1478 1759 1476    2 1244  989  215   89 1690  102 1562  563  640 1687
 1027 1185  705 1771  213  655  416   42   85 1199  897 1714  771 1920
 1542 1924  839  790 1336 1233   81 1067  169  821  955 1543 1879 1156
  996  185  436  313  182  650  682  697 1126  549 1231 1969 1525 1907
 1540  489]


In [28]:
"""
Visualize decoded sentences.
"""

which = 991
print("Decoded phoneme sequence: ", decodedOut[which])
print("Ground truth phoneme sequence: ", phonemizedSentences[9000 + which])
print("Ground truth label: ", LABELS[9000 + which])
print(" ")
print("Levenshtein distance between decoded and ground truth sequence: ", Levenshtein.distance(decodedOut[which], phonemizedSentences[9000 + which]))
print("Length of ground truth sequence: ", len(phonemizedSentences[9000 + which]))

Decoded phoneme sequence:  ['DH', 'EY', 'G', 'AA', 'T', 'AH', 'N', 'AY', 'S', 'W', 'AH', 'N']
Ground truth phoneme sequence:  ['DH', 'EY', 'V', 'G', 'AA', 'T', 'AH', 'N', 'AY', 'S', 'W', 'AH', 'N']
Ground truth label:  theyve got a nice one
 
Levenshtein distance between decoded and ground truth sequence:  1
Length of ground truth sequence:  13


In [29]:
import torch, k2
from pathlib import Path
import sys

ICEFALL_ROOT = Path("/mnt/dataDrive/emgFullCorpora/toUpload/Icefall/").resolve()
if str(ICEFALL_ROOT) not in sys.path:
    sys.path.insert(0, str(ICEFALL_ROOT))


from icefall.lexicon import Lexicon
from icefall.decode import get_lattice, one_best_decoding
from icefall.utils import get_texts

In [ ]:
"""
HLG is created using the following steps.
We use LibriSpeech-100 ranscriptions to create the graph. It has approximalety 38000 sentences and 35000 unique words. 

1) python prepare_lang.py --lang-dir DATA/ckptsLargeVocab/lang_phone --debug false (this expects lexicon.txt)

2a) /mnt/dataDrive/kenLM/kenlm/build/bin/lmplz -o 4   < DATA/ckptsLargeVocab/libri100.txt > DATA/ckptsLargeVocab/lm/4gram.arpa
 b)  python3 -m kaldilm   --read-symbol-table DATA/ckptsLargeVocab/lang_phone/words.txt   --disambig-symbol '#0' --max-order 4   DATA/ckptsLargeVocab/lm/4gram.arpa > DATA/ckptsLargeVocab/lm/G_4_gram.fst.txt

3) python prepare_lang_fst.py   --lang-dir DATA/ckptsLargeVocab/lang_phone   --has-silence 1   --ngram-G DATA/ckptsLargeVocab/lm/G_4_gram.fst.txt
4) python compile_hlg.py   --lang-dir DATA/ckptsLargeVocab/lang_phone   --lm G_4_gram
"""

In [30]:
langDir = "DATA/ckptsLargeVocab/lang_phone"
device = "cpu"  


def readSymbolTable(path):
    s2i, i2s, maxId = {}, {}, -1
    with open(path, "r", encoding = "utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            s, i = line.split()
            i = int(i)
            s2i[s] = i
            i2s[i] = s
            if i > maxId:
                maxId = i
    return s2i, i2s, maxId + 1 

def safeLoadFsaDict(ptPath):
    try:
        d = torch.load(ptPath, map_location = "cpu", weights_only = True)
    except TypeError:
        d = torch.load(ptPath, map_location = "cpu")
    except Exception:
        d = torch.load(ptPath, map_location = "cpu", weights_only = False)
    return d

def buildAcousticTokenIds(tok2id, VMinus1):
    EXCLUDE = {
        "<eps>", "<blk>",
        "SPN", "NSN", "<SPOKEN_NOISE>", "<UNK>", "<NOISE>", "<SIL>", "<NOISE_SIG>"
    }
    items = [(sym, tid) for sym, tid in tok2id.items()
             if sym not in EXCLUDE and not sym.startswith("#") and sym != "<eps>"]

    items.sort(key = lambda x: x[1])

    if len(items) < VMinus1:
        found = ", ".join(sym for sym, _ in items)
        raise RuntimeError(
            f"Found only {len(items)} acoustic tokens, need {VMinus1}.\n"
            f"Found: {found}\n"
            f"Tip: If your training used a different phone inventory, add those symbols back."
        )
    items = items[:VMinus1]
    return [tid for _, tid in items]



langPath = Path(langDir)
assert langPath.exists(), f"Not found: {langPath}"

tok2id, id2tok, CTok = readSymbolTable(langPath / "tokens.txt")
w2id, id2word, CWord = readSymbolTable(langPath / "words.txt")

lex = Lexicon(langDir)

HLGDict = safeLoadFsaDict(langPath / "HLG.pt")
HLG = k2.Fsa.from_dict(HLGDict)
HLG = k2.arc_sort(HLG).to(device).requires_grad_(False)


In [31]:
DECODED_WORDS = []
for i in range(len(outs)):
    print(i)
    logitsORlogprobsTV = torch.as_tensor(outs[i][:int(testInputLengths[i]), :], dtype = torch.float32)
    assert logitsORlogprobsTV.ndim == 2, logitsORlogprobsTV.shape
    T, V = logitsORlogprobsTV.shape
    assert V >= 2, "Expect at least 2 classes (phones + blank)."
    BLANK_CLASS = V - 1  

    nnetOutput = logitsORlogprobsTV.unsqueeze(0).to(device)

    acousticTokenIds = buildAcousticTokenIds(tok2id, VMinus1 = V-1)
    assert len(acousticTokenIds) == V-1

    
    NEG_INF = -1e30
    N = nnetOutput.shape[0]
    nnetLogitsToken = torch.full(
        (N, T, CTok), NEG_INF, dtype = nnetOutput.dtype, device = nnetOutput.device
    )

    nnetLogitsToken[..., 0] = nnetOutput[..., BLANK_CLASS]

    for clsIdx, tokId in enumerate(acousticTokenIds):
        nnetLogitsToken[..., tokId] = nnetOutput[..., clsIdx]

    nnetLogprobsToken = nnetLogitsToken
    supervisionSegments = torch.tensor([[0, 0, T]], dtype = torch.int32, device = "cpu")
    HLG = HLG.to(nnetLogprobsToken.device)
    
    lattice = get_lattice(
    nnet_output = nnetLogprobsToken,
    decoding_graph = HLG,
    supervision_segments = supervisionSegments,
    search_beam = 50,
    output_beam = 50,
    min_active_states = 30,
    max_active_states = 10000)

    best = one_best_decoding(lattice, use_double_scores = True)
    idsList = get_texts(best) 
    decodedWords = [id2word[i] for i in idsList[0] if i in id2word]
    DECODED_WORDS.append(decodedWords)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
import jiwer

WER = []
for i in range(len(DECODED_WORDS)):
    ref = LABELS[9000 + i].lower()
    hyp = " ".join(DECODED_WORDS[i]).lower()

    werScore = jiwer.wer(ref, hyp)
    WER.append(werScore)

In [33]:
print(np.mean(WER))

0.7315237173993518


In [36]:
np.save("wer.npy", WER)

In [34]:
indices = np.argsort(np.array(levs)/np.array(phoneLENGTHS))
print(indices[:100])

[ 374  452 1619   59  991  669 1214  536 1491  706 1222  202 1751 1346
 1767  170  564  693 1381 1179   87  288 1119  841 1135  778  173  852
 1620 1555 1898  762 1341  602 1807 1633  620  810  813 1056  973  797
 1478 1759 1476    2 1244  989  215   89 1690  102 1562  563  640 1687
 1027 1185  705 1771  213  655  416   42   85 1199  897 1714  771 1920
 1542 1924  839  790 1336 1233   81 1067  169  821  955 1543 1879 1156
  996  185  436  313  182  650  682  697 1126  549 1231 1969 1525 1907
 1540  489]


In [35]:
which = 991
print(" ".join(DECODED_WORDS[which]).lower())
print(LABELS[9000 + which])

they got as one
theyve got a nice one
